In [ ]:
# This script is just used to test the convolution operation, ignore it.









import numpy as np
from scipy.signal import convolve2d

# Define the 2D matrix
A = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])

# Define the 2D kernel
K = np.array([[0, 1, 0],
              [1, -4, 1],
              [0, 1, 0]])

# Pad the array to simulate periodic boundaries
pad_width = (len(K) - 1) // 2
A_padded = np.pad(A, pad_width, mode='wrap')

# Perform the convolution
result = convolve2d(A_padded, K, mode='valid')

# Crop to the original size
result = result[:A.shape[0], :A.shape[1]]
print(result)


[[ 12   9   6]
 [  3   0  -3]
 [ -6  -9 -12]]


In [10]:
print(pad_width)
A_padded = np.pad(A, pad_width+1, mode='wrap')
print(A_padded)
print(convolve2d(A_padded, K, mode='valid'))

1
[[5 6 4 5 6 4 5]
 [8 9 7 8 9 7 8]
 [2 3 1 2 3 1 2]
 [5 6 4 5 6 4 5]
 [8 9 7 8 9 7 8]
 [2 3 1 2 3 1 2]
 [5 6 4 5 6 4 5]]
[[-12  -6  -9 -12  -6]
 [  6  12   9   6  12]
 [ -3   3   0  -3   3]
 [-12  -6  -9 -12  -6]
 [  6  12   9   6  12]]


In [1]:
import torch
import torch.nn.functional as F

def dynamic_conv2d(input_tensor, kernel_generator):
    # 动态生成卷积核
    dynamic_kernel = kernel_generator(input_tensor)
    # 应用动态卷积核
    output = F.conv2d(input_tensor, dynamic_kernel, padding=1)
    return output


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

def target_age_coe(target_age, kernel_size):
    # target_age: torch.tensor, shape [1, 1, N*N]
    # return a tensor R_tensor with shape [1, kernel_size*kernel_size, N*N], where for each i in range(kernel_size),
    # R_tensor[0, 0, i, :] = target_age[0, 0, 0, :]
    # age_coe = balabalabala...
    age_coe = torch.rand_like(target_age)
    R_tensor = torch.zeros(1,kernel_size*kernel_size , target_age.shape[2])
    for i in range(target_age.shape[2]):
        R_tensor[0, :, i] = age_coe[0, 0, i].item()
    return R_tensor

test = np.random.rand(1, 1, 10)
test = torch.tensor(test).float()

result_numpy = target_age_coe(test, 3).numpy()
print(result_numpy.shape)
print(result_numpy[:, :, 0:3])


def distance_coe(distance_kernel, data_shape):
    # distance_kernel: torch.tensor, shape [kernel_size, kernel_size]
    # data_shape: tuple, [1, kernel_size*kernel_size, N*N]
    # flat the distance_kernel to a 1D tensor with shape [kernel_size * kernel_size, 1]
    flatted_distance_kernel = distance_kernel.view(-1, 1)
    print(distance_kernel)
    print(flatted_distance_kernel)
    # return a tensor R_tensor with shape [1, 1, kernel_size*kernel_size, N*],
    #  where for each j in range(N*N),
    # R_tensor[0, 0, :, j] = flatted_distance_kernel
    R_tensor = torch.zeros(data_shape)
    # row = torch.ones(1, data_shape[2])
    for i in range(data_shape[1]):
        R_tensor[0, i, :] = flatted_distance_kernel[i, 0].item()
    return R_tensor



    # R_tensor = torch.zeros(data_shape)
    # for j in range(data_shape[3]):
    #     R_tensor[0, :, j] = flatted_distance_kernel[:, 0]

#test for distance_coe
# test = np.random.rand(3, 3)
# test_data_shape = (1, 9, 100)
# test = torch.tensor(test).float()
# result_numpy = distance_coe(test, test_data_shape).numpy()
# print(result_numpy.shape)
# print(result_numpy[0, :, 0:3])



    

def infection_time_coe(infection_time):
    # infection_time: torch.tensor, shape [1, kernel_size*kernel_size, N]
    # return a random tensor R_tensor with the same shape as infection_time
    # infection_time_coe = balabalabala...
    R_tensor = torch.rand_like(infection_time)
    return R_tensor

# test infection_time_coe
# test = np.random.rand(1, 1, 3, 10)
# test = torch.tensor(test).float()
# result_numpy = infection_time_coe(test).numpy()
# print(result_numpy)

def infected_p_matrix(input_tensor, kernel_size, spread_p):
    # shape of input_tensor: [batch_size, channels, height, width]
    # should be: [1, 3, N, N]
    # 3 denote state_matrix, target_age, infection_time
    batch_size, channels, height, width = input_tensor.shape

    spread_scope = kernel_size // 2
    kernel = np.ones((2*spread_scope+1, 2*spread_scope+1))
    # change the kernel to distance_coefficient, 1 for nearest neighbors, 0.5 for second nearest neighbors
    for kernel_layer in range(spread_scope):
        kernel[0+kernel_layer, 0+kernel_layer: 2*spread_scope + 1 - kernel_layer] = 2**(kernel_layer-spread_scope)
        kernel[-1-kernel_layer, 0+kernel_layer: 2*spread_scope + 1 - kernel_layer] = 2**(kernel_layer-spread_scope)
        kernel[0+kernel_layer: 2*spread_scope + 1 - kernel_layer, 0+kernel_layer] = 2**(kernel_layer-spread_scope)
        kernel[0+kernel_layer: 2*spread_scope + 1 - kernel_layer, -1 - kernel_layer] = 2**(kernel_layer-spread_scope)
        # kernel[-1-kernel_layer, 0+kernel_layer: -1 - kernel_layer] = 2**(kernel_layer-spread_scope)
        # kernel[0+kernel_layer: -1 - kernel_layer, 0+kernel_layer] = 2**(kernel_layer-spread_scope)
        # kernel[0+kernel_layer: -1 - kernel_layer, -1 - kernel_layer] = 2**(kernel_layer-spread_scope)
        # kernel[-1-kernel_layer, :] = 2**(kernel_layer-spread_scope)
        # kernel[:, 0+kernel_layer] = 2**(kernel_layer-spread_scope)
        # kernel[:, -1-kernel_layer] = 2**(kernel_layer-spread_scope)
    kernel[spread_scope, spread_scope] = 0

    kernel = torch.tensor(kernel).float()

    # padding the input tensor with periodic boundary condition
    padded_input = F.pad(input_tensor, (kernel_size // 2, kernel_size // 2, kernel_size // 2, kernel_size // 2), mode='circular')
    # test
    print(padded_input)
    # padded_input = F.pad(input_tensor, (kernel_size // 2, kernel_size // 2,
    #                                     kernel_size // 2, kernel_size // 2), mode='constant', value=1.0)

    # unfold the padded input tensor to get all the windows
    unfolded = F.unfold(padded_input, kernel_size=kernel_size)  # [batch_size, channels * kernel_size * kernel_size, num_windows]
    # unfolded[0, :kernel_size * kernel_size, :] should be the unfolded window of state_matrix
    # unfolded[0, kernel_size * kernel_size: 2*kernel_size * kernel_size, :] should be the unfolded window of target_age
    # unfolded[0, 2*kernel_size * kernel_size:, :] should be the unfolded window of infection_time
    # we need to find out the list of j ensure that unfolded[0, kernel_size//2, j] == -1 
    # (such that the center of the window is healthy tree)
    sub_tensor =  unfolded[0, kernel_size * kernel_size // 2, :]
    indices = torch.nonzero(sub_tensor == -1).squeeze()
    # test
    print(indices)
    unfolded[:,  :kernel_size*kernel_size , :] = 0
    unfolded[:,  :kernel_size*kernel_size , indices.squeeze()] = spread_p
    # test
    print(unfolded)
    # calculate the distance_coe
    distance_coe_tensor = distance_coe(kernel, unfolded[:, :kernel_size*kernel_size , :].shape)
    # calculate the infection_time_coe
    infection_time_coe_tensor = infection_time_coe(unfolded[:, 2*kernel_size*kernel_size:, :])
    # calculate the target_age_coe
    target_age_coe_tensor = target_age_coe(unfolded[:, kernel_size * kernel_size + kernel_size * kernel_size//2, :] , kernel_size)
    coe_matrix = 1 - distance_coe_tensor * target_age_coe_tensor * infection_time_coe_tensor*unfolded[:,  :kernel_size*kernel_size , :]
    product = coe_matrix.prod(dim=1)
    infected_p = 1 -  product.view(batch_size, 1, height, width)
    print('shape of infected_p:', infected_p.size())
    # generate a random matrix with the same shape as input_tensor to simulate the infection process,
    # its value should belong to [0, 1]
    random_p = torch.rand_like(infected_p)
    # filter out the infected trees by comparing the random matrix with infected_p
    infected_matrix = (random_p < infected_p).float()
    # intersection of infected trees and healthy trees
    intersection = (input_tensor[:, 0, :, :] == -1).float() * infected_matrix
    # update the input_tensor
    input_tensor[intersection == 1] = 1
    return input_tensor


def SpreadDisease(forest, age_list, infection_time, pSpread):
    """
    Function to propagate the disease on a populated forest.
    
    Parameters
    ==========
    forest : 2-dimensional array. -1 for healthy trees, 1 for infected trees, 0 for empty cells.
    pSpread : Probability of spreading the disease.
    """
    infected_index = np.where(forest == 1)
    infected_positions_i = infected_index[0]
    infected_positions_j = infected_index[1]


    # Changing value of infected_state and healthy_state might cause big problem!!!!!
    spread_scope = 3  # scope of spreading the disease          !!! for spread_scope = 2!!!
    infected_state = 1 # state of infected trees
    healthy_state = -1 # state of healthy trees
    empty_state = 0 # state of empty cells

    # bias to avoid log(0) problem
    bias = 1e-10

    tensor_forest = torch.tensor(forest).float() # N*N
    tensor_age = torch.tensor(age_list).float() # N*N
    tensor_infection_time = torch.tensor(infection_time).float() # N*N
    tensor_input = torch.stack([tensor_forest, tensor_age, tensor_infection_time], dim=1) # [3, N, N]
    # the input for the function should be [1, 3, N, N]
    tensor_input = tensor_input.unsqueeze(0)
    # update the forest
    updated_forest = infected_p_matrix(tensor_input, 2*spread_scope+1, pSpread)
    # # filter of infected trees
    # infected_matrix = np.zeros(forest.shape)
    # infected_matrix[infected_positions_i, infected_positions_j] = infected_state
    # # add periodic boundary conditions to badding
    # padded_infected_matrix = np.pad(infected_matrix, spread_scope , mode='wrap')
    # # process to calculate the probability that the center tree get infected
    # p_matrix = np.log(1 - padded_infected_matrix*pSpread + bias)
    # kernel = np.ones((2*spread_scope+1, 2*spread_scope+1))
    # # change the kernel to distance_coefficient, 1 for nearest neighbors, 0.5 for second nearest neighbors
    # for kernel_layer in range(spread_scope):
    #     kernel[0+kernel_layer, 0+kernel_layer: 2*spread_scope + 1 - kernel_layer] = 2**(kernel_layer-spread_scope)
    #     kernel[-1-kernel_layer, 0+kernel_layer: 2*spread_scope + 1 - kernel_layer] = 2**(kernel_layer-spread_scope)
    #     kernel[0+kernel_layer: 2*spread_scope + 1 - kernel_layer, 0+kernel_layer] = 2**(kernel_layer-spread_scope)
    #     kernel[0+kernel_layer: 2*spread_scope + 1 - kernel_layer, -1 - kernel_layer] = 2**(kernel_layer-spread_scope)
    #     # kernel[-1-kernel_layer, 0+kernel_layer: -1 - kernel_layer] = 2**(kernel_layer-spread_scope)
    #     # kernel[0+kernel_layer: -1 - kernel_layer, 0+kernel_layer] = 2**(kernel_layer-spread_scope)
    #     # kernel[0+kernel_layer: -1 - kernel_layer, -1 - kernel_layer] = 2**(kernel_layer-spread_scope)
    #     # kernel[-1-kernel_layer, :] = 2**(kernel_layer-spread_scope)
    #     # kernel[:, 0+kernel_layer] = 2**(kernel_layer-spread_scope)
    #     # kernel[:, -1-kernel_layer] = 2**(kernel_layer-spread_scope)
    # kernel[spread_scope, spread_scope] = 0

    # p_matrix = convolve2d(p_matrix, kernel, mode='valid')
    # p_matrix = 1 - np.exp(p_matrix)
    # # generate random matrix to compare with the probability matrix
    # random_matrix = np.random.rand( p_matrix.shape[0], p_matrix.shape[1])
    # # determine the infected trees
    # infected_matrix[random_matrix < p_matrix] = infected_state
    # # intersection of infected trees and healthy trees
    # infected_matrix = infected_matrix * (forest == healthy_state)
    # # update the forest
    # forest[infected_matrix == infected_state] = infected_state
    return updated_forest[0, ].numpy()



    
    
    

    # # we onle need to operate on data index where unfolded[0, 0, kernel_size//2, i] == -1
    # index_health = (unfolded[0, 0, kernel_size * kernel_size // 2, :] == -1)

    


    #print the size of unfolded
    # print(unfolded.size())




    # # 对每个窗口的数值进行乘积操作
    # product = unfolded.prod(dim=1)  # 在窗口维度上按列求乘积，结果 [batch_size * channels, num_windows]

    # # 将结果恢复为原始张量的形状
    # output = product.view(batch_size, 1, height, width)
    # return output

input = np.random.rand(1, 3, 3000, 3000)
input_tensor = torch.tensor(input).float()

# 示例运行
# input_tensor = torch.tensor([[[[1.0, 2.0, 3.0],
#                                [4.0, 5.0, 6.0],
#                                [7.0, 8.0, 9.0]]]])

kernel_size = 3
output_tensor = window_product(input_tensor, kernel_size)

output = output_tensor.numpy()

print("Input Tensor:")
print(input.shape)
print("Output Tensor (Product of Each Window):")
print(output.shape)


(1, 9, 10)
[[[0.73880047 0.10181695 0.39260387]
  [0.73880047 0.10181695 0.39260387]
  [0.73880047 0.10181695 0.39260387]
  [0.73880047 0.10181695 0.39260387]
  [0.73880047 0.10181695 0.39260387]
  [0.73880047 0.10181695 0.39260387]
  [0.73880047 0.10181695 0.39260387]
  [0.73880047 0.10181695 0.39260387]
  [0.73880047 0.10181695 0.39260387]]]
torch.Size([1, 27, 9000000])
Input Tensor:
(1, 3, 3000, 3000)
Output Tensor (Product of Each Window):
(1, 1, 3000, 3000)


In [39]:
import torch

# 创建一个示例张量
tensor = torch.randint(-1, 1, (1, 1, 4, 10), dtype=torch.int)

# 获取切片 [1, 1, 4, :]
sub_tensor = tensor[0, 0, 2, :]

# 查找值为-1的元素的索引
indices = torch.nonzero(sub_tensor == -1)

# 输出索引
print(indices.squeeze())
print(tensor)
print(tensor[0,0,:, indices.squeeze()])

tensor[0, 0, :, indices.squeeze()] = 1
print(tensor)




tensor([0, 1, 4, 6])
tensor([[[[ 0,  0, -1, -1,  0, -1, -1,  0,  0, -1],
          [ 0,  0, -1, -1,  0, -1, -1, -1,  0,  0],
          [-1, -1,  0,  0, -1,  0, -1,  0,  0,  0],
          [ 0, -1, -1,  0, -1,  0, -1,  0, -1,  0]]]], dtype=torch.int32)
tensor([[ 0,  0,  0, -1],
        [ 0,  0,  0, -1],
        [-1, -1, -1, -1],
        [ 0, -1, -1, -1]], dtype=torch.int32)
tensor([[[[ 1,  1, -1, -1,  1, -1,  1,  0,  0, -1],
          [ 1,  1, -1, -1,  1, -1,  1, -1,  0,  0],
          [ 1,  1,  0,  0,  1,  0,  1,  0,  0,  0],
          [ 1,  1, -1,  0,  1,  0,  1,  0, -1,  0]]]], dtype=torch.int32)


In [46]:
tensor = torch.rand((1, 3, 10, 10), dtype=torch.float)
print(tensor.size())
kernel_size = 3
padded_tensor = F.pad(tensor, (kernel_size // 2, kernel_size // 2, kernel_size // 2, kernel_size // 2), mode='circular')
unfolded = F.unfold(padded_tensor, kernel_size=kernel_size)
print(unfolded.size())

torch.Size([1, 3, 10, 10])
torch.Size([1, 27, 100])


In [49]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9]
print(a[1:3])
print(a[3:7])

[2, 3]
[4, 5, 6, 7]
